### Imports

In [51]:
import os
import numpy as np
import pandas as pd

### Inputs

In [52]:
fusion_output_df = pd.read_csv("../outputs/fusion_output.csv")
fusion_output_class = fusion_output_df['class'].values
fusion_output_confidence = fusion_output_df['confidence'].values

In [53]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

### Kalman Filter

In [54]:
def kalman_filter(z, Q=1e-4, R=1e-2):
    n = len(z)
    xhat = np.zeros(n)
    P = np.zeros(n)
    xhat[0] = z[0]
    P[0] = 1.0
    for k in range(1, n):
        # previsão
        xhatminus = xhat[k-1]
        Pminus = P[k-1] + Q
        # atualização
        K = Pminus / (Pminus + R)
        xhat[k] = xhatminus + K * (z[k] - xhatminus)
        P[k] = (1 - K) * Pminus
    return xhat

In [55]:
confidence_fusion = np.where(fusion_output_class == 1,
    fusion_output_confidence,
    1.0 - fusion_output_confidence
)

In [56]:
kalman = kalman_filter(confidence_fusion)

# classe final a partir do filtrado
cls = (kalman >= 0.5).astype(int)

# confiança orientada à classe escolhida
conf = np.where(cls == 1, kalman, 1.0 - kalman)

In [57]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}
for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[int(cls[i])]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-89    4.9   47     0.43   0        Ruim     0.52      
-87    8.3   74     0.44   0        Ruim     0.52      
-82    6.9   98     0.34   0        Ruim     0.51      
-70    9.6   76     0.93   1        Bom      0.50      
-62    8.7   61     0.82   1        Bom      0.51      
-73    8.9   36     0.91   1        Bom      0.51      
-80    4.7   62     0.74   1        Bom      0.52      
-60    6.6   100    0.47   1        Bom      0.51      
-77    9.9   78     0.43   1        Bom      0.52      
-89    9.5   78     0.86   1        Bom      0.52      
-95    8.3   98     0.57   1        Bom      0.52      
-72    9.5   68     0.52   1        Bom      0.51      
-90    7.5   48     0.90   1        Bom      0.52      
-88    5.1   76     0.97   1        Bom      0.51      
-85    8.9   79     0.87   1        Bom      0.52      
-78    4.3   81     0.33   1        Bom      0.5

### Save

In [58]:
kalman_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
kalman_output.to_csv("../outputs/kalman_output.csv", index=False)

print("Filtro de Kalman salvo em ../outputs/kalman_output.csv")

Filtro de Kalman salvo em ../outputs/kalman_output.csv
